## Feature Engineering using pandas

### Import required packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

#### Read in the data

In [ ]:
train_dataset = pd.read_csv('train.csv')
submission_dataset = pd.read_csv('test.csv')

### Feature engineering

#### 1. Embarked

In [ ]:
train_dataset.Embarked.isnull().sum()

We have 2 rows missing a value for Embarked for train dataset.

In [ ]:
with pd.option_context('display.max_rows', None):
    display(train_dataset[train_dataset.Embarked.isnull()])


It stands to reason that people who paid a similar amount, also had a class 1 ticket and were on the same deck, embarked from the same location and a google search about the two passengers revealed that they embarked at Southampton.

In [ ]:
# Fill in missing Embarked value
train_dataset.loc[train_dataset.Embarked.isnull(), 'Embarked'] = 'S'

In [ ]:
train_dataset[train_dataset.Cabin=='B28']

In [ ]:
train_dataset.Embarked.isnull().sum()

Emarked now has no missing value.

SInce *Embarked* is categorical we need to convert it to numerical form to make it useful for our machine learning algorithm

In [ ]:
train_dataset.Embarked.unique()

It has 3 unique values which we will one-hot encode

In [ ]:
dummy = pd.get_dummies(train_dataset.Embarked)
train_dataset = pd.concat([train_dataset, dummy], axis=1).drop('Embarked', axis=1)

In [ ]:
train_dataset.rename(columns={'C':'EmbarK_Cherbourg', 'Q':'Embark_Queenstown', 'S':'Embark_Southampton'}, inplace=True)

The **Embarked** feature has been replaced with the various embarkment points with each having 1 to indicate it was the point and 0 to indicate otherwise.

#### 2. Cabin

In [ ]:
train_dataset.Cabin.isna().sum()

We have 687 missing cabin values

**i**. Using the ticket series, fare amount and Pclass am filling the missing Cabin values

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Fare < 50)]

In [ ]:
train_dataset[train_dataset.Fare==49.5042]

Passengers 494 and 711 paid same amount, belonged in same class hence should have been in the same deck 'C'

In [ ]:
train_dataset.loc[493, 'Cabin'] = 'C91'

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Fare < 50)]

In [ ]:
train_dataset[train_dataset.Fare == 30.6958]

Passengers 175 and 794 paid same amount, belonged in same class hence should have been in the samedeck 'A'

In [ ]:
train_dataset.loc[793, 'Cabin'] = 'A8'

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Fare < 50)]

In [ ]:
train_dataset[(train_dataset.Fare == 27.7208) & (train_dataset.Ticket.str.startswith('PC')) ]

Passengers who paid 27.7208 and belonged to 1st class should have been in the same deck 'B'

In [ ]:
train_dataset.loc[((train_dataset.Cabin.isna()) & (train_dataset.Fare == 27.7208) & (train_dataset.Ticket.str.startswith('PC'))), 'Cabin'] = 'B5'

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Fare < 50)]

In [ ]:
train_dataset[train_dataset.Fare == 25.925]

I will update Cabin value to E24 for Passengerid 169 beacuse of closeness in fare amount to fare amounts of 26 which were given that Cabin.

In [ ]:
train_dataset.loc[168, 'Cabin'] = 'E24'

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Fare < 50)]

In [ ]:
train_dataset[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Pclass == 1)]['Cabin'].describe(include='all').T

Replace all missing cabins with Ticket series PC and Pclass of 1 with the frequent cabin of B5

In [ ]:
train_dataset.loc[(train_dataset.Ticket.str.startswith('PC')) & (train_dataset.Pclass == 1) & (train_dataset.Cabin.isna()), 'Cabin'] = 'B5'

In [ ]:
train_dataset[(train_dataset.Pclass == 1)].describe(include='all').T

Replace all missing Cabin deck letter with B, the most occurring Cabin for Pclass 1

In [ ]:
train_dataset.loc[(train_dataset.Pclass == 1) & (train_dataset.Cabin.isna()), 'Cabin'] = 'B7'

To fill the missing cabins for 1st class i have used ticket series information, fare amount, the 1st class cabin information for other passengers.

**ii**.Handling 2nd class missng cabins

In [ ]:
with pd.option_context('display.max_rows', None):
    display(train_dataset[(train_dataset.Pclass == 2)])

Due to the high number of missing cabin values i will use ticket series and fare amount to attempt filling them

In [ ]:
train_dataset[train_dataset.Ticket.str.startswith('SC/')]

The deck for passengers with ticket series of 'SC/' will be updated to D as seen above

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('SC/')) & (train_dataset.Cabin.isna())),'Cabin'] = 'D'

In [ ]:
with pd.option_context('display.max_rows', None):
    display(train_dataset[(train_dataset.Pclass == 2)])

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('C.A')) & (train_dataset.Fare==10.5))]

Update Cabin to F33 for passengers in Pclass 2, ticket series 'C.A.' and fare amount of 10.5

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('C.A')) & (train_dataset.Fare==10.5) & (train_dataset.Cabin.isna())), 'Cabin']='F33'

Cabins for passengers with ticket series 'C.A.' has been updated to deck F

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('C.A')) & (train_dataset.Cabin.isna())), 'Cabin'] = 'F32'

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('S.O')) )]

Cabins for passengers with ticket series 'S.O.' has been updated to deck E

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('S.O')) ), 'Cabin']='E77'

In [ ]:
train_dataset.loc[((train_dataset.Ticket.str.startswith('S.C')) & (train_dataset.Pclass==2)), 'Cabin'] ='D'

Update missing cabin for remaining Pclass 2 to E

In [ ]:
train_dataset.loc[(train_dataset.Pclass == 2) & (train_dataset.Cabin.isna()), 'Cabin'] = 'E'

**iii**. Handling missing cabins for 3rd class

Update missing cabin column for Pclass 3 to G

In [ ]:
with pd.option_context('display.max_rows', None):
    display(train_dataset[(train_dataset.Pclass == 3) & (train_dataset.Cabin.isna())])

In [ ]:
train_dataset.loc[(train_dataset.Pclass == 3) & (train_dataset.Cabin.isna()), 'Cabin'] = 'G'

In [ ]:
train_dataset.Cabin.isna().sum()

The first letter of the Cabin refers to the deck hence we are going to create a new feature called deck that may have contributed to the survival rate

In [ ]:
train_dataset['Deck'] = train_dataset['Cabin'].apply(lambda x: x[0])

In [ ]:
train_dataset.Deck.value_counts()

In [ ]:
train_dataset[train_dataset.Deck == 'T']

In [ ]:
train_dataset[train_dataset.Fare == 35.5]

The decks of the ship ranged from A to G hence the value of T for deck is wrong and will be update to A since passenger who paid that amount and were in 1st class were in that deck

In [ ]:
train_dataset.loc[train_dataset.Deck == 'T', ['Cabin', 'Deck']] = 'A'

In [ ]:
train_dataset.Deck.value_counts()

We will dropthe Cabin feature

In [ ]:
train_dataset.drop('Cabin', axis=1, inplace=True)

#### 3. Sex

Convert categorical gender column to numeric through one hot encoding and drop it

In [ ]:
dummy_sex = pd.get_dummies(train_dataset.Sex)
train_dataset = pd.concat([train_dataset, dummy_sex], axis=1).drop('Sex', axis=1)
train_dataset.rename(columns={'female':'Female', 'male':'Male'}, inplace=True)
train_dataset.columns

#### 4. Sibsp & Parch

We are going to create a feature called family size that includes the two features and the person. Family size could have been a factor in the survival.

In [ ]:
train_dataset['FamilySize'] = train_dataset['SibSp'] + train_dataset['Parch'] + 1

#### 5. Name

We are going to extract the title of all passengers as it may affected the survival rate

In [ ]:
def get_title(x):
    return x.split(',')[1].split('.')[0].strip().strip()

In [ ]:
train_dataset['Title'] = train_dataset.Name.apply(get_title)

In [ ]:
train_dataset.Title.unique()

The 'Name' feature has been replaced with 'Title' so we are dropping it.

In [ ]:
train_dataset.drop('Name', axis=1, inplace=True)

#### 6. Age

Group all passengers data by its Title first, and then compute the median of each title group before eventually using these medians to fill the missing values for age

In [ ]:
median_age = train_dataset.groupby('Title')['Age'].mean()
median_age

In [ ]:
def fill_age(x):
    for index, value in zip(median_age.index, median_age.values):
        if x['Title'] == index:
            return value

In [ ]:
train_dataset['Age'] = train_dataset.apply(lambda x: fill_age(x) if np.isnan(x['Age']) else x['Age'], axis=1)

In [ ]:
train_dataset.Age.isna().sum()

In [ ]:
print(f'Maximum age:{train_dataset.Age.max()}\nMinimum age:{train_dataset.Age.min()}')

We can categorize the age column into bins

In [ ]:
groups = [0, 5, 17, 25, 50, 80]
labels = ['Infant', 'Kid', 'Young', 'Adult', 'Old']
train_dataset['Age'] = pd.cut(train_dataset['Age'], bins = groups, labels=labels)

We then one hot encode the Age feature and drop it

In [ ]:
train_dataset.Age.value_counts()

In [ ]:
dummy_age = pd.get_dummies(train_dataset.Age, prefix='', prefix_sep='')
train_dataset = pd.concat([train_dataset, dummy_age], axis=1).drop('Age', axis=1)
train_dataset

In [ ]:
with pd.option_context('display.max_columns', None):
    display(train_dataset.head())

In [ ]:
columns_to_drop = ['PassengerId','SibSp','Parch','Ticket','Fare']
train_dataset.drop(columns=columns_to_drop, inplace=True)

Drop duplicate columns

In [ ]:
train_dataset = train_dataset.T.drop_duplicates().T
train_dataset.columns

We still have Deck and Title colunms with categorical values

In [ ]:
train_dataset.Deck.unique()

In [ ]:
train_dataset.Title.unique()

In [ ]:
len(train_dataset.columns)

In [ ]:
dummy_deck = pd.get_dummies(train_dataset.Deck, prefix='Deck', prefix_sep='_')
train_dataset = pd.concat([train_dataset, dummy_deck], axis=1).drop('Deck', axis=1)

In [ ]:
dummy_title = pd.get_dummies(train_dataset.Title)
train_dataset = pd.concat([train_dataset, dummy_title], axis=1).drop(columns=['Title'])
train_dataset.columns

In [ ]:
train_dataset.to_csv('cleaned_train.csv', index=False)